## Parte 2: Recolección de información
#### Web Scraping a "Infobae" para investigación de informes referentes a riegos pre electorales

In [1]:
# Importación de librerias
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle
import pandas as pd
import time
import random

# Opciones del navegador
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--headless')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

In [2]:
# Clase para personalización
class bcolors:
    WARNING = '\033[93m'
    FAIL = '\033[91m'

# Control de errores
try:   
    # Definición de función
    def Seconds(result):
        hours = round(result/3600)
        minutes = round((result - (hours * 3600))/60)
        seconds = round(result - ((hours * 3600) + (minutes * 60)))
        if seconds < 0: seconds = seconds*-1
        if minutes < 0: minutes = minutes*-1
        final_time = "El sitema tardó: " + str(hours) + " horas " + str(minutes) + " minutos " + str(seconds) + " segundos en finalizar su ejeción"
        return final_time
    
except Exception as e: 
    print(bcolors.FAIL + "Revisa el error => ", e) # Impresión del error personalizado

In [3]:
# Definición de la clase
class IterableClass:
    
    # Función inicial o constructor
    def __init__(self, list_values = None): # None es para que sea opcional
        self.list_values = list_values
        self.dateframe = pd.read_csv(r'fraude_electoral_Infobae.csv') # lectura del archivo
        self.i = 0
        
    # Retornar un iterador
    def __iter__(self):
        return self
    
    # Retorna un elemento del contenedor
    def __next__(self):
        if self.i >= len(self.list_values):
            print("\nNo hay más valores a recorrer... ⬇")
            raise StopIteration
            
        item = self.list_values[self.i]
        self.i += 1
        return item  

In [4]:
# Instancia del objeto
IC = IterableClass()

# Acceso a la lectura del archivo Iris.csv
IC.dateframe

,Unnamed: 0,headers_text,headers_link
0,0,Campaña presidencial de Petro habría violado l...,https://www.infobae.com/colombia/2023/05/03/ca...
1,1,Código Electoral: el proyecto fue aprobado en ...,https://www.infobae.com/colombia/2023/05/17/co...
2,2,Qué es el voto electrónico mixto y por qué ha ...,https://www.infobae.com/colombia/2023/05/19/vo...
3,3,“En Colombia no hay posibilidad de fraude elec...,https://www.infobae.com/america/colombia/2022/...
4,4,Guerra de fiscales: Francisco Barbosa sería ac...,https://www.infobae.com/colombia/2023/05/16/gu...
...,...,...,...
427,427,A María Fernanda Cabal no le gustó para nada q...,https://www.infobae.com/america/colombia/2020/...
428,428,Eduardo Pulgar podría salir de La Picota este ...,https://www.infobae.com/america/colombia/2020/...
429,429,Video: Luis Carlos Vélez habló del matoneo que...,https://www.infobae.com/america/colombia/2020/...
430,430,¿Qué propone el referendo de Rodrigo Lara? La ...,https://www.infobae.com/america/colombia/2020/...


In [5]:
def JoinParagraphs(paragraphs):
    TextParagraphs = ""
    for paragraph in paragraphs:
        TextParagraphs = TextParagraphs + paragraph.text
        
    return TextParagraphs

In [6]:
def JoinThemes(themes):
    TextThemes = ""
    for theme in themes:
        TextThemes = TextThemes + theme.text + ", "
    return TextThemes

In [7]:
# Declaraciones de variables
titles = []
summaries = []
authors = []
dates = []
paragraphs = []
themes = []

# Variable para definir el inicio de tiempo en ejecución
start = time.time()

# Acceso a la página
driver = webdriver.Chrome(
    service = Service(ChromeDriverManager().install()),
    options = chrome_options
)

# Ocultamiento al navegador del uso del driver
driver = uc.Chrome()

# Asignación de dataframe a una variable
links = IC.dateframe

# Control del error al momento de finalizar el páginado
try:
    
    # Iteración para acceso a los links del dataframe
    for i in range(len(links)):
        
        news = {
            "titles" : [],
            "summaries" : [],
            "authors" : [],
            "dates" : [],
            "paragraphs" : [],
            "themes" : [],
        }
        
        # Acceso a cada link del dataframe
        driver.get(links.iloc[i]['headers_link'])

        # Títulos
        # ****************************************************************************************************************
        title = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/div/section/header/div/h1')
        if title:
            for i in title:
                print(titles.append(i.text))
        else: titles.append('...')
        # ****************************************************************************************************************

        # Resúmenes
        # ****************************************************************************************************************
        summary = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/div/section/header/div/h2')
        if summary: 
            for i in summary:
                print(summaries.append(i.text))
        else: summaries.append('...')
        # ****************************************************************************************************************

        # Autor
        # ****************************************************************************************************************
        author = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/div/section/header/div/div[2]/p/span[2]')
        if author: 
            for i in author:
                print(authors.append(i.text))
        else: authors.append('...')
        # ****************************************************************************************************************

        # Fecha
        # ****************************************************************************************************************
        date = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/div/section/article/div[1]/span')
        if date: 
             for i in date:
                print(dates.append(i.text))
        else: dates.append('...')
        # ****************************************************************************************************************

        # Parrafos
        # ****************************************************************************************************************
        paragraphs_a = driver.find_elements(By.CLASS_NAME, 'paragraph')
        if paragraphs_a: 
            paragraphs.append(JoinParagraphs(paragraphs_a))
        else: paragraphs.append('...')
        # ****************************************************************************************************************

        # Temas
        # ****************************************************************************************************************
        tags = driver.find_elements(By.CSS_SELECTOR, 'a.d23-article-tag')
        if tags: 
            themes.append(JoinThemes(tags))
        else: themes.append('...')
        # ****************************************************************************************************************
                
        # Construcción del diccionario 
        news = {
            "titles" : titles,
            "summaries" : summaries,
            "authors" : authors,
            "dates" : dates,
            "paragraphs" : paragraphs,
            "themes" : themes
        }

        # Construcción inicial del dataframe
        df = pd.DataFrame(news)

        # Modo DEBUG (como el dd de Laravel)
        # display(df)
    else:
        driver.quit();
        
except Exception as e:
    display("Exception :", e)
    driver.quit() 
except NoSuchElementException as nse:
    display("NoSuchElementException :", nse)
    driver.quit()
except TimeoutException as tim:
    display("TimeoutException :", tim)
    driver.quit()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [8]:
df = df.drop_duplicates(subset=["titles"])

In [9]:
df.to_csv("fraude_electoral_infobae_parte_2.csv")

In [10]:
# Variable para definir el final de tiempo en ejecución
end = time.time()

# Variable para calcular tiempo y para enviar a la función
result = end-start

#Impresión del tiempo que tardó
print(Seconds(result))

El sitema tardó: 4 horas 12 minutos 22 segundos en finalizar su ejeción


In [11]:
df

,titles,summaries,authors,dates,paragraphs,themes
0,Campaña presidencial de Petro habría violado l...,"El estratega español Xavier Vendrell, quien fu...",Luis Benito,"3 May, 2023","Durante la campaña presidencial de 2022, el ho...","Gustavo Petro, Campaña Presidencial, Guardiane..."
1,Código Electoral: el proyecto fue aprobado en ...,Al Proyecto de ley le hace falta dos discusion...,Julián Villamil,"17 May, 2023","Recientemente, en la plenaria del Senado se ap...","Código Electoral, Senado, Cámara de representa..."
2,Qué es el voto electrónico mixto y por qué ha ...,La plenaria del Senado aprobó la propuesta par...,...,"19 May, 2023","A diferencia de otras reformas, la electoral n...","Elecciones, Colombia, Senado, Digital, Voto el..."
3,“En Colombia no hay posibilidad de fraude elec...,Desde la Registraduría informaron que a un día...,...,"27 May, 2022",Cuando sólo faltan tres días para las eleccion...,"COLOMBIA - NOTICIAS, COLOMBIA - ACTUALIDAD, AL..."
4,Guerra de fiscales: Francisco Barbosa sería ac...,"Andrés Arauz, derrotado en el balotaje por el ...",...,"16 May, 2023",No solo en Colombia acusan al fiscal general d...,"Francisco Barbosa, Andrés Arauz, Eduardo Monte..."
...,...,...,...,...,...,...
427,A María Fernanda Cabal no le gustó para nada q...,"En entrevista con Semana, Duque habló de los ‘...",...,"19 Dic, 2020","En entrevista con Semana, el presidente de Col...","COLOMBIA-NOTICIAS, IVAN DUQUE, PREVENCION Y AC..."
428,Eduardo Pulgar podría salir de La Picota este ...,La Corte Suprema de Justicia tiene en sus mano...,...,"5 Dic, 2020","El pasado martes, primero de diciembre, Eduard...","COLOMBIA-NOTICIAS, EDUARDO PULGAR, JAIME GRANA..."
429,Video: Luis Carlos Vélez habló del matoneo que...,El periodista y director de La F.M. fue tenden...,...,"7 Nov, 2020","Este sábado, el periodista colombiano Luis Car...","COLOMBIA-NOTICIAS, LUIS CARLOS VÉLEZ, MATONEO,..."
430,¿Qué propone el referendo de Rodrigo Lara? La ...,La propuesta del senador de la República está ...,...,"13 Oct, 2020","Este martes, 13 de octubre, el senador Rodrigo...","COLOMBIA-NOTICIAS, COLOMBIA-POLÍTICA, Referend..."
